In [393]:
import pandas as pd
import numpy as np
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from nltk.corpus import wordnet as wn

In [355]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\putri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\putri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\putri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\putri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [403]:
dataset=pd.read_excel("C:/Users/putri/DataspellProjects/dsProject/Overview.xlsx")
print('data shape: ', dataset.shape)
dataset.head() # prints the first 5 rows of the dataset

data shape:  (50, 5)


,No,No. Putusan,Lembaga Peradilan,Barang Bukti,Amar_Putusan
0,1,462/Pid.Sus/2022/PN.Sda,PN Sidoarjo,1 (satu) buah pipet kaca bekas pakai sabu terd...,Menyatakan Terdakwa Kartini alias Mama Mila bi...
1,2,456/Pid.Sus/2022/PN.Sda,PN Sidoarjo,16 (enam belas) bungkus plastic klip yang dida...,Menyatakan terdakwa Duri Setiawan bin Subiyat ...
2,3,487/Pid.Sus/2022/PN Sda,PN Sidoarjo,Sebuah bungkus teh celup yang di dalamnya beri...,Menyatakan Para Terdakwa I. Rihan Iswahyudi bi...
3,4,471/Pid.Sus/2022/PN Sda,PN Sidoarjo,1 (satu) pocket plastik klip berisi Narkotika ...,Menyatakan terdakwa Muhammad Darmadi Pranggono...
4,5,512/Pid.Sus/2022/PN Sda,PN Sidoarjo,1 (satu) bungkus plastik klip ukuran kecil yan...,Menyatakan Terdakwa ALI HAMZA Bin ZULKIFLI RIT...


In [357]:
dataset.isnull().sum()

No                   0
No. Putusan          0
Lembaga Peradilan    0
Barang Bukti         0
Amar_Putusan         0
dtype: int64

In [358]:
for i,txt in enumerate(dataset['Amar_Putusan']):
    subject = re.findall('Subject:(.*\n)',txt)
    if (len(subject) !=0):
        dataset.loc[i,'Subject'] =str(i)+' '+subject[0]
    else:
        dataset.loc[i,'Subject'] ='NA'
df_news = dataset[['Subject','Amar_Putusan']]

In [359]:
print(df_news['Subject'])

0     NA
1     NA
2     NA
3     NA
4     NA
5     NA
6     NA
7     NA
8     NA
9     NA
10    NA
11    NA
12    NA
13    NA
14    NA
15    NA
16    NA
17    NA
18    NA
19    NA
20    NA
21    NA
22    NA
23    NA
24    NA
25    NA
26    NA
27    NA
28    NA
29    NA
30    NA
31    NA
32    NA
33    NA
34    NA
35    NA
36    NA
37    NA
38    NA
39    NA
40    NA
41    NA
42    NA
43    NA
44    NA
45    NA
46    NA
47    NA
48    NA
49    NA
Name: Subject, dtype: object


In [360]:
df_data =dataset[['Subject','Amar_Putusan']]

In [361]:
# Change all the text to lower case
df_data['Amar_Putusan']=[entry.lower() for entry in df_data['Amar_Putusan']]

In [362]:
## data Cleaning for content data of news
df_data.Amar_Putusan =df_data.Amar_Putusan.replace(to_replace='from:(.*\n)',value='',regex=True)
df_data.Amar_Putusan =df_data.Amar_Putusan.replace(to_replace='lines:(.*\n)',value='',regex=True)
df_data.Amar_Putusan =df_data.Amar_Putusan.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True) #remove punctuation except
df_data.Amar_Putusan =df_data.Amar_Putusan.replace(to_replace='-',value=' ',regex=True)
df_data.Amar_Putusan =df_data.Amar_Putusan.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_data.Amar_Putusan =df_data.Amar_Putusan.replace(to_replace='  ',value='',regex=True)                #remove double white space
df_data.Amar_Putusan =df_data.Amar_Putusan.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace

In [363]:
## data cleaning for subject data
df_data.Subject =df_data.Subject.replace(to_replace='Re:',value='',regex=True)
df_data.Subject =df_data.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
df_data.Subject =df_data.Subject.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_data.Subject =df_data.Subject.replace(to_replace='  ',value='',regex=True)    #remove double white space
df_data.Subject =df_data.Subject.apply(lambda x:x.strip())

In [364]:
## ## Checking  and drop empty data
for i,sen in enumerate(df_data.Amar_Putusan):
    if len(sen.strip()) ==0:
        print(str(i))
        #file_data.text[i] = np.nan
        df_data=df_data.drop(str(i),axis=0).reset_index().drop('index',axis=1)

In [365]:
# okenization : In this each entry in the file_data will be broken into set of words
df_data['Word tokenize']= [word_tokenize(entry) for entry in df_data.Amar_Putusan]

In [366]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):

        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        stop_factory = StopWordRemoverFactory()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stop_factory.get_stop_words() and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
                # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                #file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k

In [367]:
## This Function took around 13 hours for word Lemmatized and remove the Stop words & single character of word of each 11314 rows.
df_clean = wordLemmatizer(df_data['Word tokenize'][0:50])
df_clean

,Keyword_final
0,"['menyatakan', 'terdakwa', 'kartini', 'alias',..."
1,"['menyatakan', 'terdakwa', 'duri', 'setiawan',..."
2,"['menyatakan', 'terdakwa', 'rihan', 'iswahyudi..."
3,"['menyatakan', 'terdakwa', 'muhammad', 'darmad..."
4,"['menyatakan', 'terdakwa', 'ali', 'hamza', 'bi..."
5,"['menyatakan', 'terdakwa', 'najih', 'shihabfud..."
6,"['menyatakan', 'terdakwa', 'yusron', 'arizona'..."
7,"['menyatakan', 'terdakwa', 'launa', 'wahyudi',..."
8,"['menyatakan', 'terdakwa', 'suryono', 'bin', '..."
9,"['menyatakan', 'terdakwa', 'mohammad', 'nagara..."


In [368]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

In [369]:
## Insert New column in df_news to stored the Clean Keyword
df_data.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())

In [370]:
## Already Word lemmatize clean keywords the below path:-

#df_news[['Subject','Clean_Keyword']].to_json("WordLemmatize20NewsGroup.json")
df =pd.read_json('C:/Users/putri/DataspellProjects/dsProject/DatasetInformasi.json')
df_data['Clean_Keyword'] =df['Clean_Keyword']

In [371]:
df_news_save= df_data
df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
df_news_save

,Subject,Amar_Putusan
0,NA,menyatakan terdakwa kartini alias mama mila bi...
1,NA,menyatakan terdakwa duri setiawan bin subiyat ...
2,NA,menyatakan para terdakwa i. rihan iswahyudi bi...
3,NA,menyatakan terdakwa muhammad darmadi pranggono...
4,NA,menyatakan terdakwa ali hamza bin zulkifli rit...
5,NA,menyatakan terdakwa m. najih shihabfudin alias...
6,NA,menyatakan terdakwa yusron arizona alias aris ...
7,NA,menyatakan terdakwa launa wahyudi bin achamad ...
8,NA,menyatakan terdakwa suryono bin suparman telah...
9,NA,menyatakan terdakwa mohammad nagarawan dwi pra...


In [396]:
count_vectorizer = CountVectorizer()
train_vectors = count_vectorizer.fit_transform(df_data['Amar_Putusan'])
print(train_vectors[0].todense())

[[0 4 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
  0 0 0 0 0 0 0 1 2 0 0 0 0 0 2 0 0 1 0 1 0 0 0 0 2 0 0 0 1 0 1 0 1 2 0 0
  1 0 0 0 1 1 0 0 1 0 0 0 5 0 1 2 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 3 0 0 3 0 0 2 0 0 0 0 0 2 3 0 0 3 0 0 1 0 0 0 0 0 0 1 0 0 1
  0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 0 0 0 0 0 1 2 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0
  0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
  0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1
  0 0 0 1 0 1 0 0 0 0 0 0 2 0 0 0 1 1 

In [397]:
from sklearn.feature_extraction.text import TfidfVectorizer

## Create Vocabulary
vocabulary = set()

for doc in df_data.Clean_Keyword:
    vocabulary.update(doc.split(','))

vocabulary = list(vocabulary)

# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

# Fit the TfIdf model
tfidf.fit(df_data.Clean_Keyword)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(df_data.Clean_Keyword)

In [373]:
vocabulary[0:4]

['diatur', 'selamet', 'mama', 'dedik']

In [374]:
with open('C:/Users/putri/DataspellProjects/dsProject/tfid.pkl','wb') as handle:
    pickle.dump(tfidf_tran, handle)

In [375]:
### load model
t = pickle.load(open('C:/Users/putri/DataspellProjects/dsProject/tfid.pkl','rb'))

In [376]:
### Save Vacabulary
with open("C:/Users/putri/DataspellProjects/dsProject/vocab.txt", "w") as file:
    file.write(str(vocabulary))

In [377]:
### load Vacabulary
with open("C:/Users/putri/DataspellProjects/dsProject/vocab.txt", "r") as file:
    data2 = eval(file.readline())

In [378]:
print(data2)

['diatur', 'selamet', 'mama', 'dedik', 'binti', 'gold', 'sari', 'sembilan', 'rachmad', 'dijual', 'core', 'paijo', 'denda', 'umum', 'sebagaimana', 'suryono', 'jean', 'hak', 'bukan', 'danu', 'sebesar', 'damar', 'dosy', 'tosca', 'sepuluh', 'udin', 'aqua', 'andri', 'tetap', 'klip', 'mild', 'ach', 'semula', 'tujuh', 'scoopy', 'menyimpan', 'membeli', 'rifdayanto', 'bersalah', 'serta', 'warna', 'meyakinkan', 'miyanto', 'semua', 'pidana', 'pocket', 'widi', 'zuneehru', 'najih', 'launa', 'besi', 'dunhil', 'merk', 'hamza', 'tutup', 'kariyadi', 'diganti', 'sudarsono', 'la', 'telah', 'aprianto', 'dikurangkan', 'abdul', 'plasstik', 'merek', 'vivo', 'kuswanto', 'plastik', 'jutarupiah', 'grenjeng', 'suprapto', 'sebrat', 'agung', 'zainudin', 'denganberat', 'esse', 'coolmi', 'arvianto', 'dibayar', 'jinjing', 'menetapkan', 'netto', 'achamad', 'surat', 'beli', 'adi', 'menjual', 'lipat', 'diri', 'seberat', 'hijau', 'jahat', 'sutaji', 'suparman', 'sartono', 'cangklong', 'nawariyanto', 'iswahyudi', 'bungusny

In [379]:
## Create vector for Query/search keywords
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))

    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q

In [380]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [381]:
def cosine_similarity_T(k, query):
    #print("Cosine Similarity")
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    #print("\nQuery:", query)
    #print("")
    #print(tokens)

    d_cosines = []

    query_vector = gen_vector_T(q_df['q_clean'])

    for d in tfidf_tran.A:

        d_cosines.append(cosine_sim(query_vector, d))

    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    #print(out)
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Amar_Putusan'] = df_data['Amar_Putusan'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

In [383]:
%time cosine_similarity_T(10,'terdakwa')

Wall time: 18.3 ms


,index,Amar_Putusan,Score
0,8,menyatakan terdakwa suryono bin suparman telah...,0.260302
1,30,menyatakan terdakwa m. darwis bin mat bakri te...,0.246092
2,25,menyatakan terdakwa achmad choirul bin subari ...,0.243730
3,11,menyatakan terdakwa agus budi setiawan alias t...,0.233139
4,34,menyatakan terdakwa yudha kurniawan arvianto b...,0.231925
5,27,menyatakan terdakwa fery wisnu saputra alias p...,0.229880
6,48,menyatakan terdakwa aprianto hermawan alias pa...,0.228033
7,17,menyatakan terdakwa damar mada bhaskara bin an...,0.227239
8,16,menyatakan terdakwa randa prayoga bin suprapto...,0.227028
9,37,menyatakan terdakwa alvialy rifdayanto sebba b...,0.222673
